In [2]:
%load_ext autoreload
%load_ext memory_profiler 
%load_ext dotenv
%autoreload 2
%dotenv

In [3]:
import logging
from lightning import Trainer
from geovision.experiment.config import ExperimentConfig
from geovision.data.interfaces import ImageDatasetDataModule
from geovision.models.module import ClassificationModule

from geovision.io.local import FileSystemIO as fs
from geovision.experiment.loggers import (
    get_csv_logger, 
    get_ckpt_logger,
    get_classification_logger
)

In [4]:
config = ExperimentConfig.from_yaml("config.yaml")
datamodule = ImageDatasetDataModule(config.dataset_constructor, config.dataset_config, config.dataloader_config)

logger = logging.getLogger(__name__)
logging.basicConfig(
    filename = f"{fs.get_new_dir("logs")/config.project_name}.log",
    filemode = "a",
    format = "%(asctime)s : %(name)s : %(levelname)s : %(message)s",
    level = logging.INFO
)

loggers: list = list()
loggers.append(csv_logger := get_csv_logger(config))

callbacks: list = list()
callbacks.append(ckpt_logger := get_ckpt_logger(config))
callbacks.append(metrics_logger := get_classification_logger(config))

In [8]:
config = ExperimentConfig.from_yaml("config.yaml")
trainer = Trainer(logger = loggers, callbacks = callbacks, **config.trainer_params)
trainer.fit(
    model = ClassificationModule(config), 
    datamodule = datamodule, 
    ckpt_path = config.last_ckpt_path 
)

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


"found ckpts: ['epoch=2_step=771.ckpt', 'epoch=5_step=1542.ckpt', 'epoch=8_step=2313.ckpt', 'epoch=9_step=2571.ckpt', 'epoch=11_step=3084.ckpt', 'epoch=11_step=3085.ckpt', 'epoch=13_step=3599.ckpt', 'epoch=14_step=3855.ckpt', 'epoch=15_step=4113.ckpt', 'epoch=17_step=4627.ckpt', '*epoch=19_step=5141.ckpt']"

/home/sambhav/miniconda3/envs/dev/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/sambhav/experiments/imagenette/logging_test/ckpts exists and is not empty.
Restoring states from the checkpoint path at /home/sambhav/experiments/imagenette/logging_test/ckpts/epoch=19_step=5141.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | ResNet           | 11.2 M | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.716    Total estimated model params size (MB)
87        Modules in train mode
0         Modules in eval mode
Restored all states from the checkpoint at /home/sambhav/experiments/imagenette/logging_test/ckpts/epoch=19_step=5141.ckpt


Training: |          | 0/? [00:00<?, ?it/s]

/home/sambhav/miniconda3/envs/dev/lib/python3.12/site-packages/lightning/pytorch/loops/training_epoch_loop.py:161: You're resuming from a checkpoint that ended before the epoch ended and your dataloader is not resumable. This can cause unreliable results if further training is done. Consider using an end-of-epoch checkpoint or make your dataloader resumable by implementing the `state_dict` / `load_state_dict` interface.


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=22` reached.
